# Bot-E Image Meme Generation

This notebook currently focuses on creating memes for bot-e questions. Instagram requires jpg format, so this script converts the png file from the question into a jpg file for the meme.

Meme Creation Steps:

- Get url for question image you would like to create a meme for.
- Toggle settings until the meme text looks good
- post meme image to server (TODO)
- The server will post 2 memes per hour max to Meta



In [3]:
from PIL import Image, ImageDraw, ImageFont
import os
import requests
from io import BytesIO
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from ipywidgets import Layout
import ipywidgets as widgets
from IPython.display import display, clear_output

def hex_to_rgb(value):
    value = value.lstrip('#')
    length = len(value)
    return tuple(
        int(value[i:i + length // 3], 16) for i in range(0, length, length // 3)
    )


def create_meme(image_url, font_path, text1, text2=None, position='bottom', alignment='right', font_size=40, font_color='#FFFFFF', horizontal_margin=10, vertical_margin=10):
    try:
        response = requests.get(image_url, verify=False, timeout=10)  
        response.raise_for_status()  
    except requests.RequestException as e:
        print(f"Failed to retrieve the image: {e}")
        return

    img_data = BytesIO(response.content)
    img = Image.open(img_data).convert("RGBA")  
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_path, font_size)

    text_color = hex_to_rgb(font_color)

    img_width, img_height = img.size

    text_bbox = draw.textbbox((0, 0), text1, font=font)  # using textbbox instead of textsize
    text_width, text_height = text_bbox[2], text_bbox[3]
    if text2:
        text_bbox2 = draw.textbbox((0, 0), text2, font=font)  # using textbbox for second text
        text_width2, text_height2 = text_bbox2[2], text_bbox2[3]
        text_height += text_height2  

    if position == 'top':
        y = vertical_margin
    else:
        y = img_height - text_height - vertical_margin

    if alignment == 'left':
        x = horizontal_margin
    elif alignment == 'center':
        x = (img_width - text_width) / 2
    else:
        x = img_width - text_width - horizontal_margin

    draw.text((x, y), text1, font=font, fill=text_color)

    if text2:
        y += text_height - 15  # Move down for the second line of text
        text_width2, text_height2 = draw.textsize(text2, font=font)
        if alignment == 'left':
            x = horizontal_margin
        elif alignment == 'center':
            x = (img_width - text_width2) / 2
        else:
            x = img_width - text_width2 - horizontal_margin
        draw.text((x, y), text2, font=font, fill=text_color)

    # Convert the RGBA image to RGB before saving as JPEG
    rgb_img = img.convert("RGB")

    rgb_img.save('images/memes/meme.jpg')  # Save as JPEG
    rgb_img.show()

# List all files in the fonts/ directory
font_files = [f for f in os.listdir('fonts/') if f.endswith('.ttf')]

# Create a Dropdown widget with the font files
font_path_widget = widgets.Dropdown(
    options=[os.path.join('fonts/', f) for f in font_files],
    description='Font:',
)
# Define a layout for wider fields
wide_layout = Layout(width='500px')

# Now apply this layout to your widgets
image_url_widget = widgets.Text(value='https://bot-e.com/images/questions/f/fUakuRPLjyr.png', description='Image URL:', layout=wide_layout)
font_path_widget = widgets.Dropdown(options=['fonts/Lato-Regular.ttf'], description='Font Path:', layout=wide_layout)
text1_widget = widgets.Text(value='I Love', description='Text 1:', layout=wide_layout)
text2_widget = widgets.Text(value='Frogs', description='Text 2:', layout=wide_layout)
position_widget = widgets.Dropdown(options=['top', 'bottom'], value='top', description='Position:')
alignment_widget = widgets.Dropdown(options=['left', 'center', 'right'], value='left', description='Alignment:')
font_size_widget = widgets.IntSlider(value=20, min=10, max=100, description='Font Size:')
font_color_widget = widgets.ColorPicker(value='#FFFFFF', description='Font Color:')
horizontal_margin_widget = widgets.IntSlider(value=30, min=0, max=100, description='Horiz. Margin:')
vertical_margin_widget = widgets.IntSlider(value=280, min=0, max=500, description='Vert. Margin:')

# Define a button to trigger the function
run_button = widgets.Button(description='Create Meme')

# Define the output widget
output_widget = widgets.Output()

# Define the function to be triggered
def on_button_clicked(b):
    with output_widget:
        clear_output()
        create_meme(
            image_url=image_url_widget.value,
            font_path=font_path_widget.value,
            text1=text1_widget.value,
            text2=text2_widget.value,
            position=position_widget.value,
            alignment=alignment_widget.value,
            font_size=font_size_widget.value,
            font_color=font_color_widget.value,
            horizontal_margin=horizontal_margin_widget.value,
            vertical_margin=vertical_margin_widget.value
        )

# Link the button click event to the function
run_button.on_click(on_button_clicked)

# Display the widgets
widgets.VBox([
    image_url_widget,
    font_path_widget,
    text1_widget,
    text2_widget,
    position_widget,
    alignment_widget,
    font_size_widget,
    font_color_widget,
    horizontal_margin_widget,
    vertical_margin_widget,
    run_button,
    output_widget
])

